<h1> ch8 어텐션 </h1>

<p>
    RNN을 사용한 문장 생성을 했다. RNN 2개로 seq2seq를 만들어 덧셈 문제도 풀었다. 이제는 어텐션을 학습한다.
</p>

<h2> 8.1 어텐션의 구조 </h2>

<p>
    어텐션 메커니즘로 seq2seq 처럼 필요한 정보에만 '주목'할 수 있다. 게다가 seq2seq가 가진 문제를 해결할 수 있다. 
</p>

<h3> 8.1.1 seq2seq의 문제점 </h3>

<p>
    seq2seq는 Encoder가 시계열 데이터를 인코딩한다. 인코딩된 정보를 Decoder로 전달한다. Encoder의 출력은 '고정 길이의 벡터'이다. 고정 길이 벡터라 함은 입력 문장의 길이에 관계없이 항상 같은 길이의 벡터로 변환한다는 것이다. 긴 문장을 억지로 욱여넣듯이 고정 길이의 벡터로 밀어 넣는다. 
</p>

<h3> 8.1.2 Encoder 개선 </h3>

<p>
    마지막 은닉 상태만을 Decoder에 전달했다. 이것을 Encoder 출력의 길이는 입력 문장의 길이에 따라 매번 바꿔주는 것이 좋다. 예를 들어, LSTM 계층의 은닉 상태 벡터를 모두 이용하는 것이다. 모든 시각의 은닉 상태 벡터를 모두 이용한다면 입력한 단어 수의 벡터를 얻을 수 있다. LSTM 계층의 은닉 상태의 '내용'은 시각별 LSTM 계층에 직전에 입력된 단어에 대한 정보가 많이 포함되어 있다. Encoder가 출력하는 hs 행렬은 각 단어에 해당하는 벡터들의 집합이다.
</p>

<h3> 8.1.3 Decoder 개선 1</h3>

<p>
    Encoder는 각 단어에 대응하는 LSTM 계층의 은닉 상태 벡터를 hs에 모아 출력한다. 이전 seq2seq에서는 Encoder의 마지막 은닉상태 벡터만을 Decoder에 넘겼다. Decoder의 LSTM 계층의 '첫' 은닉 상태로 설정한다. 이 뜻은 현재 hs 벡터 집합에서 마지막 은닉 상태만 빼내서 Decoder에 전달한 것이다. 
</p>
<p>
    사람이 문장을 번역할 때는 '나'='I' 나 '고양이'='cat'이라는 지식을 활용한다. 어떤 단어에 주목하여 그 단어의 변환을 수시로 한다. 입력과 출력의 여러 단어 중 어떤 단어끼리 서로 관련되어 있는지를 학습 시 켜야한다. 
</p>
<p>
    '도착어 단어'와 대응 관계에 있는 '출발어 단어'의 정보를 골라내야 한다. 필요한 정보에만 주목하여 정보로부터 시계열 변환을 수행하는 것이 목표이다. 이 구조가 어텐션이다. Decoder에 LSTM 다음 계층에 '어떤 계산'을 수행하는 계층을 추가해야 한다. '어떤 계산'의 입력은 두가지, hs와 LSTM 계층의 은닉 상태이다. 여기서 필요한 정보만 골라 위쪽 Affine 계층으로 출력한다. Decoder에 입력한 단어와 대응 관계인 단어 벡터를 hs를 통해 골라내야 한다. 이러한 '선택' 작업을 통해 '어떤 계산'을 하는 것이지만, 이러한 선택하는 작업은 미분이 불가능하다. '선택한다'는 작업을 미분 가능한 연산으로 대체해야 한다. '하나를 선택'하는 것이 아니라, '모든 것을 선택'하여 각 단어의 중요도를 나타내는 '가중치'를 별도로 계산한다. 각 단어의 중요도를 나타내는 '가중치' (a) 를 이용한다. a는 확률 분포처럼 각 원소가 0.0 ~ 1.0 사이의 스칼라이며, 원소의 총합은 1이다. 가중치 a와 hs로부터 가중합을 구하여, 벡터를 얻는다. 결과 벡터를 '맥락 벡터'라고 부르고 기호는 c이다. 
</p>

In [2]:
import numpy as np

T, H = 5, 4
hs = np.random.randn(T, H)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])

ar = a.reshape(5, 1).repeat(4, axis=1)
print(ar.shape)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=0)
print(c.shape)

(5, 4)
(5, 4)
(4,)


In [3]:
# 미니배치 처리용 가중합
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
a = np.random.randn(N, T)
ar = a.reshape(N, T, 1).repeat(H,  axis=2)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=1)
print(c.shape)

(10, 5, 4)
(10, 4)


In [4]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None
        
    def forward(self, hs, a):
        N, T, H = hs.shape
        
        ar = a.reshape(N, T, 1).repeat(H, axis=2)
        t = hs * ar
        
        c = np.sum(t, axis=1)
        
        self.cache = (hs, ar)
        return c
    
    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape
        
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)
        
        return dhs, da        

<h3> 8.1.4 Decoder 개선 2 </h3>

<p>
    각 단어의 중요도를 나타내는 a가 있으면 '맥락 벡터'를 얻을 수 있다. a 또한 자동으로 학습할 수 있어야 한다. Decoder 은닉 상태 벡터와 hs의 각 단어 벡터와 얼마나 '비슷한가'를 수치로 나타낸다. 수치를 나타내는 가장 단순한 방법인 벡터의 '내적'을 이용한다. 
    $$
        \mathbf{a\cdot b} = a_1b_1 + a_2b_2 + \cdots + a_nb_n
    $$
    벡터의 내적은 두 벡터가 얼마나 같은 방향을 향하고 있는가 이다. h와 hs의 각 단어 벡터와의 유사도를 구한다. s는 그 결과고, s는 정규화하기 전 값이여서, 소프트맥스 함수로 정규화해서 적용한다.
    
</p>

In [5]:
import sys
sys.path.append('..')
from common.layers import Softmax
import numpy as np

N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
hr = h.reshape(N, 1, H).repeat(T, axis=1)

t = hs * hr
print(t.shape)

s = np.sum(t, axis=2)
print(s.shape)

softmax = Softmax()
a = softmax.forward(s)
print(a.shape)

(10, 5, 4)
(10, 5)
(10, 5)


In [6]:
import sys
sys.path.append('..')
from common.np import *
from common.layers import Softmax

class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None
        
    def forward(self, hs, h):
        N, T, H = hs.shape
        
        hr = h.reshape(N, 1, H).repeat(T, axis=1)
        t = hs * hr
        s = np.sum(t, axis=2)
        a = self.softmax.forward(s)
        
        self.cache = (hs, hr)
        return a
        
    def backward(self, da):
        hs, hr = self.cache
        N, T, H = hs.shape
        
        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)
        
        return dhs, dh

<h3> 8.1.5 Decoder 개선 3</h3>

<p>
    AttentionWeight 계층과 WeightSum 계층을 각각 구현하였고, 두 계층을 하나로 결합한다. 
</p>

In [7]:
class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None
        
    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        return out 
    
    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh

In [8]:
class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None
        
    def forward(self, hs_enc, hs_dec):
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []
        
        for t in range(T):
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_des[:, t, :])
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)
            
        return  out
    
    def backward(self, dout):
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)
        
        for t in range(T):
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:, t, :] = dh
            
        return dhs_enc, dhs_dec

<h2> 8.2 어텐션을 갖춘 seq2seq 구현 </h2>

<h3> 8.2.1 Encoder 구현 </h3>

In [9]:
import sys
sys.path.append('..')
from common.time_layers import *
from ch07.seq2seq import Encoder, Seq2seq
from ch08.attention_layer import TimeAttention

class AttentionEncoder(Encoder):
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        return hs
    
    def backward(self, dhs):
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout
    

<h3> 8.2.2 Decoder 구현 </h3>

In [10]:
class AttentionDecoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(2 * H, V) / np.sqrt(2 * H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.attention = TimeAttention()
        self.affine = TimeAffine(affine_W, affine_b)
        layers = [self.embed, self.lstm, self.attention, self.affine]
        
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, xs, enc_hs):
        h = enc_hs[:, -1]
        self.lstm.set_state(h)
        
        out = self.embed.forward(xs)
        dec_hs = self.lstm.forward(out)
        c = self.attention.forward(enc_hs, dec_hs)
        out = np.concatenate((c, dec_hs), axis=2)
        score = self.affine.forward(out)
        
        return score
    
    def backward(self, dscore):
        # 깃허브 소스 참고
        pass
    
    def generate(self, enc_hs, start_id, sample_size):
        # 깃허브 소스 참고
        pass

<h3> 8.2.3 seq2seq 구현 </h3>

In [11]:
from ch07.seq2seq import Encoder, Seq2seq

class AttentionSeq2seq(Seq2seq):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        args = vocab_size, wordvec_size, hidden_size
        self.encoder = AttentionEncoder(*args)
        self.decoder = AttentionDecoder(*args)
        self.softmax = TimeSoftmaxWithLoss()
        
        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads

<h2> 8.3 어텐션 평가 </h2>

<h3> 8.3.1 날짜 형식 변환 문제 </h3>

<p>
    다양한 날짜 형식을 표준 형식으로 변환하는 것이 목표이다. 날짜 데이터는 다양한 변형이 존재하여 변환 규칙이 나름 복잡하다. 입력과 출력 사이에 알기 쉬운 대응 관계도 있다.
</p>

<h3> 8.3.2 어텐션을 갖춘 seq2seq의 학습 </h3>

In [20]:
import sys
sys.path.append('..')
sys.path.append('../ch07')
import numpy as np
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from attention_seq2seq import AttentionSeq2seq
from ch07.seq2seq import Seq2seq
from ch07.peeky_seq2seq import PeekySeq2seq

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('date.txt')
char_to_id, id_to_char = sequence.get_vocab()

# 입력 문장 반전
x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 256
batch_size = 128
max_epoch = 10
max_grad = 5.0

model = AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
               batch_size=batch_size, max_grad=max_grad)
    
    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                   id_to_char, verbose, is_reverse=True)
    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('val acc %.3f%%' % (acc * 100))
    
model.save_params()

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 4.08
| 에폭 1 |  반복 21 / 351 | 시간 5[s] | 손실 3.09
| 에폭 1 |  반복 41 / 351 | 시간 10[s] | 손실 1.90
| 에폭 1 |  반복 61 / 351 | 시간 15[s] | 손실 1.72
| 에폭 1 |  반복 81 / 351 | 시간 21[s] | 손실 1.46
| 에폭 1 |  반복 101 / 351 | 시간 26[s] | 손실 1.19
| 에폭 1 |  반복 121 / 351 | 시간 31[s] | 손실 1.14
| 에폭 1 |  반복 141 / 351 | 시간 37[s] | 손실 1.09
| 에폭 1 |  반복 161 / 351 | 시간 42[s] | 손실 1.06
| 에폭 1 |  반복 181 / 351 | 시간 47[s] | 손실 1.04
| 에폭 1 |  반복 201 / 351 | 시간 53[s] | 손실 1.03
| 에폭 1 |  반복 221 / 351 | 시간 58[s] | 손실 1.02
| 에폭 1 |  반복 241 / 351 | 시간 63[s] | 손실 1.02
| 에폭 1 |  반복 261 / 351 | 시간 68[s] | 손실 1.01
| 에폭 1 |  반복 281 / 351 | 시간 74[s] | 손실 1.00
| 에폭 1 |  반복 301 / 351 | 시간 79[s] | 손실 1.00
| 에폭 1 |  반복 321 / 351 | 시간 84[s] | 손실 1.00
| 에폭 1 |  반복 341 / 351 | 시간 89[s] | 손실 1.00
Q 10/15/94                     
T 1994-10-15
X 1978-08-11
---
Q thursday, november 13, 2008  
T 2008-11-13
X 1978-08-11
---
Q Mar 25, 2003                 
T 2003-03-25
X 1978-08-11
---
Q Tuesday, November 22, 2016  

val acc 99.920%
| 에폭 7 |  반복 1 / 351 | 시간 0[s] | 손실 0.00
| 에폭 7 |  반복 21 / 351 | 시간 5[s] | 손실 0.00
| 에폭 7 |  반복 41 / 351 | 시간 10[s] | 손실 0.00
| 에폭 7 |  반복 61 / 351 | 시간 16[s] | 손실 0.00
| 에폭 7 |  반복 81 / 351 | 시간 21[s] | 손실 0.00
| 에폭 7 |  반복 101 / 351 | 시간 26[s] | 손실 0.00
| 에폭 7 |  반복 121 / 351 | 시간 32[s] | 손실 0.00
| 에폭 7 |  반복 141 / 351 | 시간 37[s] | 손실 0.00
| 에폭 7 |  반복 161 / 351 | 시간 42[s] | 손실 0.00
| 에폭 7 |  반복 181 / 351 | 시간 48[s] | 손실 0.00
| 에폭 7 |  반복 201 / 351 | 시간 53[s] | 손실 0.00
| 에폭 7 |  반복 221 / 351 | 시간 58[s] | 손실 0.00
| 에폭 7 |  반복 241 / 351 | 시간 64[s] | 손실 0.00
| 에폭 7 |  반복 261 / 351 | 시간 69[s] | 손실 0.00
| 에폭 7 |  반복 281 / 351 | 시간 74[s] | 손실 0.00
| 에폭 7 |  반복 301 / 351 | 시간 80[s] | 손실 0.00
| 에폭 7 |  반복 321 / 351 | 시간 85[s] | 손실 0.00
| 에폭 7 |  반복 341 / 351 | 시간 90[s] | 손실 0.00
Q 10/15/94                     
T 1994-10-15
O 1994-10-15
---
Q thursday, november 13, 2008  
T 2008-11-13
O 2008-11-13
---
Q Mar 25, 2003                 
T 2003-03-25
O 2003-03-25
---
Q Tuesday, Nov

<h2> 8.4 어텐션에 관한 남은 이야기 </h2>

<h3> 8.4.1 양반향 RNN </h3>

<p>
    LSTM의 각 시각의 은닉 상태 벡터는 hs로 모아진다. 단어의 '주변' 정보를 균형 있게 담고 싶을 것이다. LSTM을 양반향으로 처리하는 방법을 생각할 수 있다. 양방향으로 처리함으로써, 각 단어에 대응하는 은닉 상태 벡터에는 좌와 우 양쪽 방향으로부터 정보를 집약할 수 있다. 
</p>

<h3> 8.4.2 Attention 계층 사용 방법 </h3>

<p>
    Attention 계층을 LSTM 계층과 Affine 계층 사이에 삽입했다. Attention 계층의 출력이 다음 시각의 LSTM 계층에 입력 되게 연결한다. 이렇게 하면 LSTM 계층이 맥락 벡터의 정보를 이용할 수 있다. 
</p>

<h3> 8.4.3 seq2seq 심층화와 skip 연결 </h3>

<p>
    현실 어플리케이션들은 풀어야 할 문제가 휠씬 복잡하다. 어텐션을 갖춘 seq2seq에도 더 높은 표현력이 요구될 것이다. RNN 계층을 깊게 쌓는 방법이다. 층을 깊게 할 때 사용되는 중요한 기법 중 skip 연결 이라는 것이 있다. skip 연결의 접속부에서는 2개의 출력이 '더해'진다. skip 연결의 기울기가 아무런 영향을 받지 않고 모든 계층으로 흐른다. 
</p>